# Support vector machines soft margin

 - Informações sobre o conjunto de dados
    1. variance of Wavelet Transformed image (continuous)
    2. skewness of Wavelet Transformed image (continuous)
    3. curtosis of Wavelet Transformed image (continuous)
    4. entropy of image (continuous)
    5. class (integer)




In [1]:
import sys 

sys.path.append("/home/davi/PycharmProjects/Reconhecimento-de-Padroes")

from sklearn.model_selection import KFold
from Strings import string
from sklearn.model_selection import train_test_split
from numpy import zeros, identity, array, concatenate, \
    ones, ravel, dot, where, multiply
from src.Utils.utils import get_accuracy, get_data

from pandas import DataFrame
import cvxopt
import cvxopt.solvers

In [2]:
def get_p(number_lines, number_columns):
    tmp_aux = identity(number_columns)
    tmp_aux[:, number_columns-1] = 0
    
    print(tmp_aux.shape)
    P = cvxopt.matrix(tmp_aux, tc='d')
    return P

In [3]:
def get_q(number_columns):
    tmp1 = zeros((number_columns, 1))
    print(tmp1.shape)
    Q = cvxopt.matrix(tmp1, tc='d')
    
    return Q

In [4]:
def get_g(x, y, number_lines):
    if array(y, ndmin=2).shape[0] == 1: 
        tmp2 = array(y, ndmin=2).T * concatenate((x, ones((number_lines, 1))), axis=1)
    else:
        tmp2 = array(y, ndmin=2) * concatenate((x, ones((number_lines, 1))), axis=1)


    # print(tmp2.shape)
    # G = cvxopt.matrix(tmp2 * -1., tc='d')
    # return G
    
    # tmp2 = array(y, ndmin=2).T * concatenate((x, ones((number_lines, 1))), axis=1)
    G = cvxopt.matrix((tmp2 * -1.).T.tolist())

    return G

In [5]:
def get_h(number_lines):
    tmp2 = ones((number_lines, 1))
    
    print(tmp2.shape)
    H = cvxopt.matrix(tmp2 * -1., tc='d')

    return H

In [6]:
def quadratic_solver(x_train, y_train):
    
    number_lines, number_columns = x_train.shape 
    
    # bias
    number_columns += 1
    
    P = get_p(number_lines, number_columns)
    Q = get_q(number_columns)
    G = get_g(x_train, y_train, number_lines)
    H = get_h(number_lines)
    
    

    try:
        solution = cvxopt.solvers.qp(P, Q, G, H)
        best_weights = ravel(solution['x'])  
        return best_weights
    
    except ValueError as error:
        print(error)

    

    


In [7]:
def train(x_train, y_train):
    solution = quadratic_solver(x_train, y_train)
    return solution

In [8]:
def get_foward(weights, x_test):
    
    print(weights)
    print(x_test.shape)
    
    if x_test.shape[1] == array(weights[:weights.shape[0]-1], ndmin=2).shape[1]:
        H_output = dot(x_test, array(weights[:weights.shape[0]-1], ndmin=2).T)

    else:
        H_output = dot(x_test, array(weights[:weights.shape[0] - 1], ndmin=2))

    H_output += weights[weights.shape[0]-1]
    
    # H_output = dot(x_test, array(weights[:2], ndmin=2).T) + weights[2]
    # vec_sup = sum(H_output == 1.)
    return H_output


In [9]:
def predict(h, domain=None):
    if domain == [-1., 1.]:
        for i in range(len(h)):
            if h[i][0] > 0:
                h[i][0] = 1
            else:
                h[i][0] = -1

        return h
    

In [10]:
def test(weights, x_test, y_test):
    h_output = get_foward(weights, x_test)
    y_output = predict(h_output, domain=[-1., 1.])

    if y_output.shape == y_test.T.shape:
        accuracy = get_accuracy(y_output, array(y_test, ndmin=2).T)
    else:
        accuracy = get_accuracy(y_output, array(y_test, ndmin=2))
    
    return accuracy

In [11]:
data = get_data("column_2C_weka.arff", type="arff")
data = DataFrame(data)
number_lines = data.shape[0]
number_columns = data.shape[1]
X = array(data, ndmin=2)[:, :number_columns-1]
Y =   array(array(data, ndmin=2)[:, number_columns-1], ndmin=2).T
train_size = .8
test_size = .2

indices = where(Y == b'Abnormal')
Y[indices] = -1

indices = where(Y == b'Normal')
Y[indices] = 1

print(X)
print(Y)

[[63.0278175 22.55258597 39.60911701 40.47523153 98.67291675 -0.254399986]
 [39.05695098 10.06099147 25.01537822 28.99595951 114.4054254 4.564258645]
 [68.83202098 22.21848205 50.09219357 46.61353893 105.9851355
  -3.530317314]
 ...
 [61.44659663 22.6949683 46.17034732 38.75162833 125.6707246 -2.707879517]
 [45.25279209 8.693157364 41.5831264 36.55963472 118.5458418 0.214750167]
 [33.84164075 5.073991409 36.64123294 28.76764934 123.9452436
  -0.199249089]]
[[-1]
 [-1]
 [-1]
 [-1]
 [-1]
 [-1]
 [-1]
 [-1]
 [-1]
 [-1]
 [-1]
 [-1]
 [-1]
 [-1]
 [-1]
 [-1]
 [-1]
 [-1]
 [-1]
 [-1]
 [-1]
 [-1]
 [-1]
 [-1]
 [-1]
 [-1]
 [-1]
 [-1]
 [-1]
 [-1]
 [-1]
 [-1]
 [-1]
 [-1]
 [-1]
 [-1]
 [-1]
 [-1]
 [-1]
 [-1]
 [-1]
 [-1]
 [-1]
 [-1]
 [-1]
 [-1]
 [-1]
 [-1]
 [-1]
 [-1]
 [-1]
 [-1]
 [-1]
 [-1]
 [-1]
 [-1]
 [-1]
 [-1]
 [-1]
 [-1]
 [-1]
 [-1]
 [-1]
 [-1]
 [-1]
 [-1]
 [-1]
 [-1]
 [-1]
 [-1]
 [-1]
 [-1]
 [-1]
 [-1]
 [-1]
 [-1]
 [-1]
 [-1]
 [-1]
 [-1]
 [-1]
 [-1]
 [-1]
 [-1]
 [-1]
 [-1]
 [-1]
 [-1]
 [-1]
 [-1]

In [12]:
for realization in range(5):
    x_train, x_test, y_train, y_test = train_test_split(X, Y, test_size=test_size)
    
    weights = train(x_train, y_train)
    # print(weights)
    if weights is None:
        print("ERROR")
        continue
    accuracy = test(weights, x_test, y_test)
    
    print(string.RUN.format(realization, None, accuracy, None))    

(7, 7)
(7, 1)
(248, 1)
     pcost       dcost       gap    pres   dres
 0:  5.0664e-04  4.6420e+02  2e+03  3e+00  3e+04
 1:  2.4923e-03  4.3471e+02  1e+03  2e+00  2e+04
 2:  1.3874e-03  8.6166e+02  1e+03  2e+00  2e+04
 3:  1.2858e-05  3.2327e+03  1e+03  1e+00  2e+04
 4:  1.5489e-06  1.0634e+04  1e+03  1e+00  1e+04
 5:  5.7470e-07  3.8326e+04  3e+03  1e+00  1e+04
 6:  3.5824e-07  1.4068e+05  1e+04  1e+00  1e+04
 7:  3.5410e-07  4.6461e+05  3e+04  1e+00  1e+04
 8:  2.2496e-06  2.0972e+06  1e+05  1e+00  1e+04
 9:  9.4598e-05  1.4121e+07  5e+05  1e+00  1e+04
10:  5.3293e-03  2.4835e+08  4e+06  1e+00  1e+04
11:  5.6862e-03  1.5022e+10  2e+07  1e+00  1e+04
Terminated (singular KKT matrix).
[-6.15668931e-02  6.15610301e-02  1.90010338e-06  6.15799632e-02
 -2.52091918e-06  1.22974238e-05 -1.21027915e-04]
(62, 6)

            Realização: 0 

            Epocas: None 

            Acuracia: 0.27419354838709675 

            Matriz de confusão: None

    
          
(7, 7)
(7, 1)
(248, 1)
     pc